# Function Calling

**Function calling** é, de fato, uma estratégia de fornecer ao LLM um “esqueleto” estruturado (normalmente em JSON Schema) descrevendo:

- Nome da função 
- O que ela faz 
- Quais argumentos aceita
- Tipo/formato desses argumentos
- Quais campos são obrigatórios

Isso guia o modelo a:
- Entender quando usar a função
- Como montar corretamente os argumentos
- Devolver uma saída estruturada e válida, reduzindo erros

Ou seja: não é para o LLM executar a função, e sim para pedir corretamente que ela seja executada pelo seu código.

In [95]:
import os, json
from dotenv import dotenv_values
import requests
from openai import OpenAI

config = dotenv_values("../.env")
openai_api_key = config["OPENAI_API_KEY"]
open_weather_key = config["OPEN_WEATHER"]

## Testando API Open Weather

In [96]:
lat = -4.35795
lon = -39.3024
url = (
    f"https://api.openweathermap.org/data/3.0/onecall"
    f"?lat={lat}&lon={lon}"
    f"&appid={open_weather_key}"
    f"&units=metric&lang=pt_br"
) 


resp = requests.get(url)  
resp.status_code

200

In [97]:
result = json.loads(resp.content)
result


{'lat': -4.358,
 'lon': -39.3024,
 'timezone': 'America/Fortaleza',
 'timezone_offset': -10800,
 'current': {'dt': 1769033264,
  'sunrise': 1768984707,
  'sunset': 1769029091,
  'temp': 27.27,
  'feels_like': 28.19,
  'pressure': 1009,
  'humidity': 57,
  'dew_point': 18.01,
  'uvi': 0,
  'clouds': 54,
  'visibility': 10000,
  'wind_speed': 2.36,
  'wind_deg': 64,
  'wind_gust': 5.38,
  'weather': [{'id': 803,
    'main': 'Clouds',
    'description': 'nublado',
    'icon': '04n'}]},
 'minutely': [{'dt': 1769033280, 'precipitation': 0},
  {'dt': 1769033340, 'precipitation': 0},
  {'dt': 1769033400, 'precipitation': 0},
  {'dt': 1769033460, 'precipitation': 0},
  {'dt': 1769033520, 'precipitation': 0},
  {'dt': 1769033580, 'precipitation': 0},
  {'dt': 1769033640, 'precipitation': 0},
  {'dt': 1769033700, 'precipitation': 0},
  {'dt': 1769033760, 'precipitation': 0},
  {'dt': 1769033820, 'precipitation': 0},
  {'dt': 1769033880, 'precipitation': 0},
  {'dt': 1769033940, 'precipitation': 

In [98]:
result["current"]["weather"][0]["description"]

'nublado'

## Functions que serão utilizadas pelo LLM

In [99]:
def get_weather(lat:float, lon:float, api_key=open_weather_key) -> str:

    """ 
    Essa função nos informa o clima  do ponto dado por 
    latitude=lat e longitude=lon no momento exato em que é chamada
    """

    if not api_key:
        return {"error": "OpenWeather api key not set"}
    
    url = (
    f"https://api.openweathermap.org/data/3.0/onecall"
    f"?lat={lat}&lon={lon}"
    f"&appid={open_weather_key}"
    f"&units=metric&lang=pt_br"
    )

    response = requests.get(url)
    result = json.loads(response.content)

    return f"O clima em lat={lat} e lon={lon}, região da {result["timezone"]}, é {result["current"]["weather"][0]["description"]}"

print(get_weather(8.36666, 115.15))


O clima em lat=8.36666 e lon=115.15, região da Asia/Manila, é nublado


In [100]:
def send_email_stup(to:str, subject:str, body:str) -> dict:
    """Stub simples: em prod troque por SendGrid/Mailgun/SMTP real."""
    return {"sent": True, "to": to, "subject": subject}


## Schema para o modelo (Function Calling)

In [101]:
functions = [
    {   
        "type": "function",
        "name": "get_weather",
        "description": "Retorna temperatura e descrição do tempo para uma cidade.",
        "parameters": {
            "type": "object",
            "properties": {
                "lat": {
                    "type": "number",
                    "minimum": -90,
                    "maximum": 90,
                    "description": "Latitude (ex: -3.171722)"},
                "lon": {
                    "type": "number",
                    "minimum":-180,
                    "maximum":180,
                    "description": "Longitude (ex: -38.5434)"},
                "api_key": {"type": "string"}
            },
            "required": ["lat", "lon"]
        }
    },
    {   
        "type": "function",
        "name": "send_email",
        "description": "Envia um email simples.",
        "parameters": {
            "type": "object",
            "properties": {
                "to": {"type": "string"},
                "subject": {"type": "string"},
                "body": {"type": "string"}
            },
            "required": ["to", "subject", "body"]
        }
    }
]


## Workflow

In [ ]:
input = (
    "Verifique o tempo em Fortaleza hoje. "
    "Se estiver chuvoso, envie um email para iran@example.com pedindo que leve guarda-chuva."
)

client_openai = OpenAI(api_key=openai_api_key)

response = client_openai.responses.parse(
    model="gpt-4.1-mini",
    input=input,
    temperature=0.0, 
    tools=functions
)

In [106]:
response

ParsedResponse[NoneType](id='resp_0eb89b6b74f4d1a60069714e31d5d08195ac8a3abbb66b1756', created_at=1769033265.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-mini-2025-04-14', object='response', output=[ParsedResponseFunctionToolCall(arguments='{"lat":-3.71722,"lon":-38.5434,"api_key":"your_openweather_api_key"}', call_id='call_WiRjMP3AJHtpogIzmcNAHOOn', name='get_weather', type='function_call', id='fc_0eb89b6b74f4d1a60069714e32a4148195a0701994f0c82663', status='completed', parsed_arguments=None)], parallel_tool_calls=True, temperature=0.0, tool_choice='auto', tools=[FunctionTool(name='get_weather', parameters={'type': 'object', 'properties': {'lat': {'type': 'number', 'minimum': -90, 'maximum': 90, 'description': 'Latitude (ex: -3.171722)'}, 'lon': {'type': 'number', 'minimum': -180, 'maximum': 180, 'description': 'Longitude (ex: -38.5434)'}, 'api_key': {'type': 'string'}}, 'required': ['lat', 'lon', 'api_key'], 'additionalProperties': False}, str